In [3]:
import sys,os, time

#SET gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '1' 


from prediction_imports import *
from imports import *
import matplotlib.pyplot as plt

Version:  2.3.2
Eager mode:  True
Version:  2.3.2
Eager mode:  True
GPU name:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [4]:
#GET THE CONFIG
configfile = "../data/CTNOAA_minorAUG.json"
with open(configfile) as f:
    config = json.load(f)

for k in config.keys():
    exec(k+'=config["'+k+'"]')


In [5]:
base_model =  custom_resunet((TARGET_SIZE[0], TARGET_SIZE[1], N_DATA_BANDS),
                        FILTERS,
                        nclasses=[NCLASSES+1 if NCLASSES==1 else NCLASSES][0],
                        kernel_size=(KERNEL,KERNEL),
                        strides=STRIDE,
                        dropout=DROPOUT,#0.1,
                        dropout_change_per_layer=DROPOUT_CHANGE_PER_LAYER,#0.0,
                        dropout_type=DROPOUT_TYPE,#"standard",
                        use_dropout_on_upsampling=USE_DROPOUT_ON_UPSAMPLING,#False,
                        )



In [6]:
# get the weights
weights = "../data/CTNOAA_minorAUG.h5"
base_model.load_weights(weights)
base_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 1024, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 1024, 7) 91          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 768, 1024, 7) 28          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 768, 1024, 7) 28          input_1[0][0]                    
_______________________________________________________________________________________

In [7]:
#define the sorting function



def AvgConf(x):
    #sort the inputs
    sorted_x = tf.sort(x, axis=-1, direction='DESCENDING', name=None)
    #take first
    TopPred = sorted_x[:,:,:,0]
    #calulate confidence
    LConfidence = 1 - TopPred
    #mean over iamge
    #conf_x = LConfidence
    conf_x = tf.reduce_mean(LConfidence, axis = 1)
    conf_x = tf.reduce_mean(conf_x, axis = 1)

    return(conf_x)

#margin output too

In [8]:
AverageConf = tf.keras.layers.Lambda(AvgConf)(base_model.output)


model = tf.keras.Model(base_model.input, AverageConf) 
#############
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 1024, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 1024, 7) 91          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 768, 1024, 7) 28          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 768, 1024, 7) 28          input_1[0][0]                    
_______________________________________________________________________________________

In [29]:
#the next steps for active learnign are to feed many images into the model (with a generator) 
# and then rank them based on confidence and margin.

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# build the generator.

#The PIL thing image file issue
#https://stackoverflow.com/questions/48395076/keras-giving-oserror-image-file-is-truncated-26-bytes-not-processed

from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None


# here are the numbers for each storms, if we want to break those out for par. processing.
#(vs. running entire dir of all storms).
# Barry, 8900
# Delta, 9785
# Dorian, 24,749
# Florence, 31,434
# Gordon, 2019
# Isaias, 8938
# Laura, 24,165
# Michael, 9600
# Sally, 6390
# Zeta, 8644
# Ida
# Henri
# pred_data_dir = '/data/NOAA_ERI/Florence'


#directory
pred_data_dir = '/data/NOAA_ERI'

batch_size = 32

#set Image size based on config
imsize = (TARGET_SIZE[0], TARGET_SIZE[1])

#rescale the images
pred_datagen = ImageDataGenerator(preprocessing_function = standardize)

# Flow images in batches 
pred_generator = pred_datagen.flow_from_directory(pred_data_dir,
                                                    batch_size = batch_size,
                                                    class_mode = None,
                                                    target_size = imsize,
                                                    shuffle = False)
#reset the generator counter
pred_generator.reset()


Found 135245 images belonging to 13 classes.


In [10]:
#set number of images based on output of previous cell
numberofimages = 135245

workfolk = 24

#make the predictions
prediction = model.predict(pred_generator,
                           verbose=1,
                           steps=numberofimages/batch_size,
                           workers = workfolk,
                           max_queue_size = 6 * workfolk)

4227/4226 [==============================] - 44031s 10s/step


In [12]:
import pandas as pd

#get the filenames
filenames = pred_generator.filenames

#convert the predictions to a list
PredList = prediction.tolist()

#and make a dataframe with filenames adn predictions
PredDF = pd.DataFrame({'Filename': filenames,'MeanConfidence': PredList})

In [13]:
PredDF.head()

,Filename,MeanConfidence
0,Barry/20190716a_jpgs/jpgs/C19748935.jpg,0.312211
1,Barry/20190716a_jpgs/jpgs/C19748941.jpg,0.328417
2,Barry/20190716a_jpgs/jpgs/C19748947.jpg,0.360161
3,Barry/20190716a_jpgs/jpgs/C19748953.jpg,0.357218
4,Barry/20190716a_jpgs/jpgs/C19748959.jpg,0.375271


In [15]:
#new csv to save
newpredcsv = '../data/LambdaSegALDF.csv'

PredDF.to_csv(newpredcsv, index=False)